In [47]:
import numpy as np
from patsy import dmatrix, EvalEnvironment

# Define a simple raw polynomial function.
def poly(x, degree, raw=True):
    # Return columns: x, x^2, ..., x^degree
    # x should be a numpy array.
    x = np.asarray(x)
    return np.column_stack([x**i for i in range(1, degree+1)])

# Create an evaluation environment that includes our poly function.
env = EvalEnvironment({'poly': poly})


In [70]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.preprocessing import scale, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from tensorflow import keras
from patsy import dmatrix
from sklearn.model_selection import KFold
# For quantile regression (placeholder use)
import statsmodels.api as sm
from statsmodels.regression.quantile_regression import QuantReg

# Global parameters
c_param = 0.5
alpha_param = 0.1
tol = 1e-6
l = 0.1  # dictionary weight for intercept

##########################
# Basic Helper Functions #
##########################

def two_norm(x):
    return np.linalg.norm(x)

def b(d, z):
    # returns vector: [1, d, (elements of z)]
    z = np.array(z).flatten()
    return np.concatenate(([1], [d], z))

def b2(d, z):
    # returns vector: [1, d, (elements of z), d*z (elementwise)]
    z = np.array(z).flatten()
    return np.concatenate(([1], [d], z, d * z))

def b_PD(d, z):
    # identity for PD elasticity (here simply returns d)
    return np.array([d])

def m_ATE(y, d, z, gamma_func):
    # returns gamma(1,z) - gamma(0,z)
    return gamma_func(1, z) - gamma_func(0, z)

def m_PD(y, x, x_up, x_down, delta, gamma_func):
    # elasticity moment function
    return (gamma_func(x_up, 0) - gamma_func(x_down, 0)) / delta

def m2(y, d, z, gamma_func):
    return y * gamma_func(d, z)

def get_p(args, est_type, X, b_func):
    if est_type == "ATE":
        return len(b_func(args['T'][0], X[0, :]))
    else:
        return X.shape[1]

def get_n(args, est_type, X):
    if est_type == "ATE":
        return len(args['T'])
    else:
        return X.shape[0]

def psi_tilde(args, est_type, y, z, alpha, gamma_func):
    if est_type == "ATE":
        # here args is a dict with a scalar 'T'
        d = args['T']
        return m_ATE(y, d, z, gamma_func) + alpha(d, z) * (y - gamma_func(d, z))
    else:
        x_up = args['X.up']
        x_down = args['X.down']
        delta = args['delta']
        return m_PD(y, z, x_up, x_down, delta, gamma_func) + alpha(z, 0) * (y - gamma_func(z, 0))

def psi_tilde_bias(args, est_type, y, z, alpha, gamma_func):
    if est_type == "ATE":
        d = args['d']
        return m_ATE(y, d, z, gamma_func)
    else:
        return m_PD(y, z, args['X.up'], args['X.down'], args['delta'], gamma_func)

def get_MNG(args, est_type, Y, X, b_func):
    if est_type == "ATE":
        T = args['T']
        n_nl = len(T)
        p = len(b_func(T[0], X[0, :]))
        B = np.zeros((n_nl, p))
        M = np.zeros((p, n_nl))
        N = np.zeros((p, n_nl))
        for i in range(n_nl):
            B[i, :] = b_func(T[i], X[i, :])
            # here we use the dictionary function in place of gamma
            M[:, i] = m_ATE(Y[i], T[i], X[i, :], b_func)
            N[:, i] = m2(Y[i], T[i], X[i, :], b_func)
        M_hat = np.mean(M, axis=1)
        N_hat = np.mean(N, axis=1)
        G_hat = (B.T @ B) / n_nl
        return M_hat, N_hat, G_hat, B
    else:
        X_up = args['X.up']
        X_down = args['X.down']
        delta = args['delta']
        n = X.shape[0]
        p = X.shape[1]
        M = np.zeros((p, n))
        N = np.zeros((p, n))
        for i in range(n):
            M[:, i] = (X_up[i, :] - X_down[i, :]) / delta
            N[:, i] = Y[i] * X[i, :]
        M_hat = np.mean(M, axis=1)
        N_hat = np.mean(N, axis=1)
        G_hat = (X.T @ X) / n
        return M_hat, N_hat, G_hat, X

#############################
# Learning Algorithm Blocks #
#############################

# RMD_dantzig: placeholder using quantile regression approach.
def RMD_dantzig(M, G, D, lambda_val=0, sparse=True):
    p = G.shape[1]
    zp = np.zeros(p)
    L_vec = np.concatenate(([l], np.ones(p-1)))
    A = np.linalg.solve(np.diag(D), G)
    R_mat = np.vstack([A, -A])
    a = np.linalg.solve(np.diag(D), M)
    r_vec = np.concatenate([a - lambda_val * L_vec, -a - lambda_val * L_vec])
    # Placeholder: in R the code uses rq.fit.sfn; here we simply return a dummy least–squares solution.
    beta_dummy = np.linalg.lstsq(G, M, rcond=None)[0].flatten()
    return {'coefficients': beta_dummy}

def RMD_lasso(M, G, D, lambda_val=0, control=None, beta_start=None):
    if control is None:
        control = {'maxIter': 1000, 'optTol': 1e-5, 'zeroThreshold': 1e-6}
    p = G.shape[1]
    L_vec = np.concatenate(([l], np.ones(p-1)))
    lambda_vec = lambda_val * L_vec * D
    if beta_start is None:
        beta = np.zeros(p)
    else:
        beta = beta_start.copy()
    wp = [beta.copy()]
    mm = 1
    while mm < control['maxIter']:
        beta_old = beta.copy()
        for j in range(p):
            rho = M[j] - np.dot(G[j, :], beta) + G[j, j] * beta[j]
            z_val = G[j, j]
            if np.isnan(rho):
                beta[j] = 0
                continue
            if rho < -lambda_vec[j]:
                beta[j] = (rho + lambda_vec[j]) / z_val
            elif abs(rho) <= lambda_vec[j]:
                beta[j] = 0
            elif rho > lambda_vec[j]:
                beta[j] = (rho - lambda_vec[j]) / z_val
        wp.append(beta.copy())
        if np.sum(np.abs(beta - beta_old)) < control['optTol']:
            break
        mm += 1
    beta[np.abs(beta) < control['zeroThreshold']] = 0
    return {'coefficients': beta, 'coef_list': wp, 'num_it': mm}

def get_D(args, est_type, Y, X, m_func, rho_hat, b_func):
    n = get_n(args, est_type, X)
    p = get_p(args, est_type, X, b_func)
    df_mat = np.zeros((p, n))
    if est_type == "ATE":
        T = args['T']
        for i in range(n):
            b_val = b_func(T[i], X[i, :])
            dot_val = np.dot(rho_hat, b_val)
            m_val = m_ATE(Y[i], T[i], X[i, :], b_func)
            df_mat[:, i] = b_val * dot_val - m_val
    else:
        X_up = args['X.up']
        X_down = args['X.down']
        delta = args['delta']
        for i in range(n):
            dot_val = np.dot(rho_hat, X[i, :])
            m_val = m_PD(Y[i], X[i, :], X_up[i, :], X_down[i, :], delta, b_func)
            df_mat[:, i] = X[i, :] * dot_val - m_val
    df_mat = df_mat ** 2
    D2 = np.mean(df_mat, axis=1)
    return np.sqrt(D2)

def RMD_stable(args, est_type, Y, X, p0, D_LB, D_add, max_iter, b_func, is_alpha=True, is_lasso=True):
    p = get_p(args, est_type, X, b_func)
    n = get_n(args, est_type, X)
    # Low-dimensional moments: use the first p0 columns of X.
    X0 = X[:, :p0]
    if est_type == "ATE":
        args0 = {'T': args['T']}
    else:
        args0 = {'X.up': args['X.up'][:, :p0], 'X.down': args['X.down'][:, :p0], 'delta': args['delta']}
    M_hat0, N_hat0, G_hat0, _ = get_MNG(args0, est_type, Y, X0, b_func)
    rho_hat0 = np.linalg.solve(G_hat0, M_hat0)
    if len(rho_hat0) < p:
        rho_hat = np.concatenate([rho_hat0, np.zeros(p - len(rho_hat0))])
    else:
        rho_hat = rho_hat0
    beta_hat0 = np.linalg.solve(G_hat0, N_hat0)
    if len(beta_hat0) < p:
        beta_hat = np.concatenate([beta_hat0, np.zeros(p - len(beta_hat0))])
    else:
        beta_hat = beta_hat0

    M_hat, N_hat, G_hat, _ = get_MNG(args, est_type, Y, X, b_func)
    lambda_val = c_param * norm.ppf(1 - alpha_param / (2 * p)) / np.sqrt(n)
    k = 1
    if is_alpha:
        diff_rho = 1
        while diff_rho > tol and k <= max_iter:
            rho_hat_old = rho_hat.copy()
            D_hat_rho = get_D(args, est_type, Y, X, m_func=m_ATE, rho_hat=rho_hat_old, b_func=b_func)
            D_hat_rho = np.maximum(D_LB, D_hat_rho) + D_add
            if is_lasso:
                rho_hat = RMD_lasso(M_hat, G_hat, D_hat_rho, lambda_val)['coefficients']
            else:
                rho_hat = RMD_dantzig(M_hat, G_hat, D_hat_rho, lambda_val)['coefficients']
            diff_rho = two_norm(rho_hat - rho_hat_old)
            k += 1
        print("k:", k)
        return rho_hat
    else:
        diff_beta = 1
        while diff_beta > tol and k <= max_iter:
            beta_hat_old = beta_hat.copy()
            D_hat_beta = get_D(args, est_type, Y, X, m_func=m2, rho_hat=beta_hat_old, b_func=b_func)
            D_hat_beta = np.maximum(D_LB, D_hat_beta) + D_add
            if is_lasso:
                beta_hat = RMD_lasso(N_hat, G_hat, D_hat_beta, lambda_val)['coefficients']
            else:
                beta_hat = RMD_dantzig(N_hat, G_hat, D_hat_beta, lambda_val)['coefficients']
            diff_beta = two_norm(beta_hat - beta_hat_old)
            k += 1
        print("k:", k)
        return beta_hat

###########################
# Stage 1 & Stage 2 Steps #
###########################

arg_Forest = {'clas_nodesize': 1, 'reg_nodesize': 5, 'ntree': 1000, 'na_action': 'omit', 'replace': True}
arg_Nnet = {'size': 8, 'maxit': 1000, 'decay': 0.01, 'MaxNWts': 10000, 'trace': False}

def get_stage1(args, est_type, Y, X, p0, D_LB, D_add, max_iter, b_func, alpha_estimator=1, gamma_estimator=1):
    p = get_p(args, est_type, X, b_func)
    n = get_n(args, est_type, X)
    MNG = get_MNG(args, est_type, Y, X, b_func)
    B = MNG[3]
    # alpha estimator
    if alpha_estimator == 0:
        rho_hat = RMD_stable(args, est_type, Y, X, p0, D_LB, D_add, max_iter, b_func, is_alpha=True, is_lasso=False)
        def alpha_hat(d, z):
            return np.dot(b_func(d, z), rho_hat)
    elif alpha_estimator == 1:
        rho_hat = RMD_stable(args, est_type, Y, X, p0, D_LB, D_add, max_iter, b_func, is_alpha=True, is_lasso=True)
        def alpha_hat(d, z):
            return np.dot(b_func(d, z), rho_hat)
    # gamma estimator
    if gamma_estimator == 0:
        beta_hat = RMD_stable(args, est_type, Y, X, p0, D_LB, D_add, max_iter, b_func, is_alpha=False, is_lasso=False)
        def gamma_hat(d, z):
            return np.dot(b_func(d, z), beta_hat)
    elif gamma_estimator == 1:
        beta_hat = RMD_stable(args, est_type, Y, X, p0, D_LB, D_add, max_iter, b_func, is_alpha=False, is_lasso=True)
        def gamma_hat(d, z):
            return np.dot(b_func(d, z), beta_hat)
    elif gamma_estimator == 2:
        # Random Forest approach
        rf = RandomForestRegressor(n_estimators=arg_Forest['ntree'],
                                   min_samples_leaf=arg_Forest['reg_nodesize'],
                                   bootstrap=arg_Forest['replace'], random_state=0)
        rf.fit(B, Y)
        def gamma_hat(d, z):
            return rf.predict(b_func(d, z).reshape(1, -1))[0]
    elif gamma_estimator == 3:
        # Neural Net via MLPRegressor
        scaler_B = StandardScaler().fit(B)
        NN_B = scaler_B.transform(B)
        scaler_Y = StandardScaler().fit(Y.reshape(-1, 1))
        NN_Y = scaler_Y.transform(Y.reshape(-1, 1)).flatten()
        nn = MLPRegressor(hidden_layer_sizes=(arg_Nnet['size'],),
                          max_iter=arg_Nnet['maxit'],
                          alpha=arg_Nnet['decay'],
                          random_state=0)
        nn.fit(NN_B, NN_Y)
        def gamma_hat(d, z):
            test = b2(d, z).reshape(1, -1)
            test_scaled = scaler_B.transform(test)
            NN_Y_hat = nn.predict(test_scaled)
            Y_hat = NN_Y_hat * scaler_Y.scale_[0] + scaler_Y.mean_[0]
            return Y_hat[0]
    elif gamma_estimator == 4:
        # 2-layer NN using Keras
        scaler_B = StandardScaler().fit(B)
        NN_B = scaler_B.transform(B)
        scaler_Y = StandardScaler().fit(Y.reshape(-1, 1))
        NN_Y = scaler_Y.transform(Y.reshape(-1, 1)).flatten()
        model = keras.models.Sequential([
            keras.layers.Dense(8, activation="relu", input_shape=(NN_B.shape[1],)),
            keras.layers.Dense(8, activation="relu"),
            keras.layers.Dense(1)
        ])
        model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
        model.fit(NN_B, NN_Y, epochs=100, batch_size=1, verbose=0)
        def gamma_hat(d, z):
            test = b(d, z).reshape(1, -1)
            test_scaled = scaler_B.transform(test)
            NN_Y_hat = model.predict(test_scaled, verbose=0)
            Y_hat = NN_Y_hat * scaler_Y.scale_[0] + scaler_Y.mean_[0]
            return Y_hat[0][0]
    return alpha_hat, gamma_hat

def rrr(args, est_type, Y, X, p0, D_LB, D_add, max_iter, dict_func, alpha_estimator, gamma_estimator, bias):
    if est_type == 'ATE':
        T = args['T']
    else:
        # For PD
        X_up = args['X.up']
        X_down = args['X.down']
        delta = args['delta']
    n = X.shape[0]
    L_folds = 5
    kf = KFold(n_splits=L_folds, shuffle=True, random_state=0)
    Psi_tilde = []
    fold_index = 0
    for i in np.arange(0, 4):
        
        index = folds_info.iloc[:, i].dropna().astype(int).tolist()
        total_indices = np.arange( 0, 9848 )
        test_index = np.delete( total_indices, index, axis = 0 )
        train_index = index

        fold_index += 1
        Y_l = Y[test_index]
        Y_nl = Y[train_index]
        X_l = X[test_index, :]
        X_nl = X[train_index, :]
        if est_type == "ATE":
            T_arr = np.array(args['T'])
            T_l = T_arr[test_index]
            T_nl = T_arr[train_index]
            n_l = len(T_l)
            stage1_args = {'T': T_nl}
        else:
            X_up_full = args['X.up']
            X_down_full = args['X.down']
            X_up_l = X_up_full[test_index, :]
            X_up_nl = X_up_full[train_index, :]
            X_down_l = X_down_full[test_index, :]
            n_l = X_l.shape[0]
            stage1_args = {'X.up': X_up_nl, 'X.down': args['X.down'][train_index, :], 'delta': args['delta']}
        alpha_hat, gamma_hat = get_stage1(stage1_args, est_type, Y_nl, X_nl, p0, D_LB, D_add, max_iter,
                                          dict_func, alpha_estimator, gamma_estimator)
        print("fold:", fold_index)
        Psi_tilde_l = []
        for i in range(n_l):
            if est_type == "ATE":
                psi_args = {'T': T_l[i]}
            else:
                psi_args = {'X.up': X_up_l[i, :], 'X.down': X_down_l[i, :], 'delta': args['delta']}
            if bias:
                psi_val = psi_tilde_bias(psi_args, est_type, Y_l[i], X_l[i, :], None, gamma_hat)
            else:
                psi_val = psi_tilde(psi_args, est_type, Y_l[i], X_l[i, :], alpha_hat, gamma_hat)
            Psi_tilde_l.append(psi_val)
        Psi_tilde.extend(Psi_tilde_l)
    Psi_tilde = np.array(Psi_tilde)
    ate = np.mean(Psi_tilde)
    Psi_infl = Psi_tilde - ate
    var = np.mean(Psi_infl ** 2)
    se = np.sqrt(var / n)
    if est_type == "ATE":
        T_arr = np.array(args['T'])
        treated = np.sum(T_arr == 1)
        untreated = np.sum(T_arr == 0)
        out = (treated, untreated, ate, se)
    else:
        out = (n, ate, se)
    return out

def printer(spec1, est_type):
    if est_type == "ATE":
        print(f" treated: {spec1[0]} untreated: {spec1[1]}   ATE: {round(spec1[2],2)}   SE: {round(spec1[3],2)}")
    else:
        print(f"   n: {spec1[0]}   AD: {round(spec1[1],2)}   SE: {round(spec1[2],2)}")

def for_tex(spec1, est_type):
    if est_type == "ATE":
        print(f" & {spec1[0]} & {spec1[1]}   & {round(spec1[2],2)}   & {round(spec1[3],2)}")
    else:
        print(f" & {spec1[0]} & {round(spec1[1],2)}   & {round(spec1[2],2)}")

#########################################
# Data Import & Processing (get_data)   #
#########################################

def get_data(df, spec, quintile, est_type="ATE"):
    if est_type == "ATE":
        Y = df["net_tfa"].values
        D = df["e401"].values
        # Low–dim specification
        X_L = df[["age", "inc", "educ", "fsize", "marr", "twoearn", "db", "pira", "hown"]].values
        # High–dim: use polynomial expansions for some variables
        poly_age = PolynomialFeatures(degree=6, include_bias=False)
        poly_inc = PolynomialFeatures(degree=8, include_bias=False)
        poly_educ = PolynomialFeatures(degree=4, include_bias=False)
        poly_fsize = PolynomialFeatures(degree=2, include_bias=False)
        age_poly = poly_age.fit_transform(df[["age"]].values)
        inc_poly = poly_inc.fit_transform(df[["inc"]].values)
        educ_poly = poly_educ.fit_transform(df[["educ"]].values)
        fsize_poly = poly_fsize.fit_transform(df[["fsize"]].values)
        X_H = np.hstack([age_poly, inc_poly, educ_poly, fsize_poly, df[["marr", "twoearn", "db", "pira", "hown"]].values])
        # Very high–dim: using patsy (remove intercept later)
        formula = "poly(age,6,raw=True) + poly(inc,8,raw=True) + poly(educ,4,raw=True) + poly(fsize,2,raw=True) + marr + twoearn + db + pira + hown"
        X_vH = dmatrix(formula, df, return_type='dataframe').values[:, 1:]
        if spec == 1:
            X = X_L
        elif spec == 2:
            X = X_H
        else:
            X = X_vH
        X = scale(X)
        n = X.shape[0]
        # Impose common support via a logistic model (no intercept)
        clf = LogisticRegression(fit_intercept=False, multi_class='multinomial', solver='lbfgs')
        clf.fit(X, D)
        p1 = clf.predict_proba(X)[:, 1]
        mask_D1 = (D == 1)
        if np.sum(mask_D1) == 0:
            min_p1, max_p1 = 0, 1
        else:
            min_p1 = p1[mask_D1].min()
            max_p1 = p1[mask_D1].max()
        indexes_to_drop = np.where((p1 < min_p1) | (p1 > max_p1))[0]
        if len(indexes_to_drop) == 0:
            indexes_to_drop = np.array([n])  # no dropping
        mask = np.ones(n, dtype=bool)
        mask[indexes_to_drop] = False
        Y_trimmed = Y[mask]
        D_trimmed = D[mask]
        X_trimmed = X[mask, :]
        if spec == 1:
            inc = X_trimmed[:, 1]
        else:
            inc = X_trimmed[:, 7] if X_trimmed.shape[1] > 7 else X_trimmed[:, 1]
        if quintile > 0:
            q = pd.qcut(inc, 5, labels=False) + 1  # quintiles 1,...,5
            mask_q = (q == quintile)
            Y_q = Y_trimmed[mask_q]
            D_q = D_trimmed[mask_q]
            X_q = X_trimmed[mask_q, :]
        else:
            Y_q, D_q, X_q = Y_trimmed, D_trimmed, X_trimmed
        return Y_q, D_q, X_q
    else:
        # PD type
        N = df.shape[0]
        cols_to_log = ["gas", "price", "income", "age", "distance"]
        for col in cols_to_log:
            df[col] = np.log(df[col])
        Y = df["gas"].values
        df["age2"] = df["age"] ** 2
        df["income2"] = df["income"] ** 2
        df_up = df.copy()
        df_down = df.copy()
        prices = df["price"].values
        delta = np.std(prices) / 4
        df_up["price"] = prices + delta / 2
        df_down["price"] = prices - delta / 2
        df["price2"] = prices ** 2
        df_up["price2"] = df_up["price"] ** 2
        df_down["price2"] = df_down["price"] ** 2
        # A simplified specification using patsy:
        formula = "price + price2 + urban + youngsingle + distance + I(distance**2) + age + age2 + income + income2"
        regressors = dmatrix(formula, df, return_type='dataframe').values
        regressors_up = dmatrix(formula, df_up, return_type='dataframe').values
        regressors_down = dmatrix(formula, df_down, return_type='dataframe').values
        if quintile > 0:
            q = pd.qcut(df["income"], 5, labels=False) + 1
            mask_q = (q == quintile)
            Y_q = Y[mask_q]
            regressors_q = regressors[mask_q, :]
            regressors_up_q = regressors_up[mask_q, :]
            regressors_down_q = regressors_down[mask_q, :]
        else:
            Y_q, regressors_q = Y, regressors
            regressors_up_q, regressors_down_q = regressors_up, regressors_down
        return Y_q, regressors_q, regressors_up_q, regressors_down_q, delta

#################################
# Main Execution of the Program #
#################################

In [71]:
df = pd.read_stata("/Users/ar8787/Dropbox/data_eco/Udesa/mater_thesis/raw/18515_Data_and_Programs/tutorial/sipp1991.dta")
spec = 1
quintile = 0
data = get_data(df, spec, quintile, est_type="ATE")
Y = data[0]
T_var = data[1]
X = data[2]
dict_func = b
p = len(b(T_var[0], X[0, :]))
args_main = {'T': T_var}

/Users/ar8787/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_polynomial.py:555: RuntimeWarning: overflow encountered in multiply
  np.multiply(
/var/folders/r4/lwrzx5hs7d3cxk4p5zsy7zqh0000gp/T/ipykernel_92557/1517224467.py:9: RuntimeWarning: overflow encountered in power
  return np.column_stack([x**i for i in range(1, degree+1)])
/Users/ar8787/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:261: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/Users/ar8787/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_data.py:280: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/Users/ar8787/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1237: FutureWarning:

In [74]:
p0 = int(np.ceil(p / 4))
if p > 60:
    p0 = int(np.ceil(p / 40))
D_LB = 0
D_add = 0.2
max_iter = 10
np.random.seed(1)
est_type = "ATE"
alpha_estimator = 1  # 0: dantzig, 1: lasso
gamma_estimator = 1  # 0: dantzig, 1: lasso, 2: rf, 3: nn, 4: 2-layer nn (works for ATE)
bias = 0  # 0: unbiased, 1: biased
results = rrr(args_main, est_type, Y, X, p0, D_LB, D_add, max_iter,
              dict_func, alpha_estimator, gamma_estimator, bias)

k: 7
k: 11
fold: 1
k: 7
k: 11
fold: 2
k: 7
k: 10
fold: 3
k: 7
k: 11
fold: 4


In [75]:
results

(3682, 6166, 5549.890651442612, 1497.4701731744835)

In [18]:
folds_info = pd.read_csv('/Users/ar8787/Dropbox/data_eco/Udesa/mater_thesis/data/folds_data.csv')

In [61]:
folds_info.iloc[:, 4].dropna().tolist()

array([6736., 5918., 1012., ..., 4138., 8829., 5305.])

In [ ]:
index

In [69]:
i = 3
index = folds_info.iloc[:, i].dropna().astype(int).tolist()
total_indices = np.arange( 0, 9848 )
test_index = np.delete( total_indices, index, axis = 0 )

In [31]:
dfaux = pd.DataFrame({'index': np.arange( 0, 9848 )})

In [ ]:
if __name__ == "__main__":
# Choose type: "ATE" for average treatment effect or "PD" for elasticity
type_ = "PD"  
for quintile in [0]:
    print("quintile:", quintile)
    if type_ == "ATE":
        df = pd.read_stata("sipp1991.dta")
        spec = 1
        data = get_data(df, spec, quintile, type_="ATE")
        Y = data[0]
        T_var = data[1]
        X = data[2]
        dict_func = b
        p = len(b(T_var[0], X[0, :]))
        args_main = {'T': T_var}
    else:
        df = pd.read_stata("gasoline_final_tf1.dta")
        spec = 1
        data = get_data(df, spec, quintile, type_="PD")
        Y = data[0]
        X = data[1]
        X_up = data[2]
        X_down = data[3]
        delta = data[4]
        p = X.shape[1]
        dict_func = b_PD
        args_main = {'X.up': X_up, 'X.down': X_down, 'delta': delta}
    p0 = int(np.ceil(p / 4))
    if p > 60:
        p0 = int(np.ceil(p / 40))
    D_LB = 0
    D_add = 0.2
    max_iter = 10
    np.random.seed(1)
    alpha_estimator = 1  # 0: dantzig, 1: lasso
    gamma_estimator = 1  # 0: dantzig, 1: lasso, 2: rf, 3: nn, 4: 2-layer nn (works for ATE)
    bias = 0  # 0: unbiased, 1: biased
    results = rrr(args_main, type_, Y, X, p0, D_LB, D_add, max_iter,
                  dict_func, alpha_estimator, gamma_estimator, bias)
    printer(results, type_)
    for_tex(results, type_)